# Trajectory picture

Note: Running Simulation and Digitization in the same notebook is fragile. Restart after each and only run the specific cells.

In [1]:
import os
import ROOT

Welcome to JupyROOT 6.16/00


In [2]:
def simulation():
    parfile = "output/traj.para.root"
    outfile = "output/traj.simu.root"

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("ERROR")

    vmcworkdir = os.environ["VMCWORKDIR"]
    os.environ["GEOMPATH"] = vmcworkdir + "/geometry"
    os.environ["CONFIG_DIR"] = vmcworkdir + "/gconfig"
    os.environ["PHYSICSLIST"] = "QGSP_INCLXX_HP"

    # Initialize Simulation
    run = ROOT.FairRunSim()
    run.SetName("TGeant4")
    run.SetStoreTraj(True)
    run.SetMaterials("media_r3b.geo")

    # Output
    output = ROOT.FairRootFileSink(outfile)
    run.SetSink(output)

    # Primary Generator
    generator = ROOT.FairPrimaryGenerator()
    box = ROOT.FairBoxGenerator(2112, 1)
    box.SetXYZ(0, 0, 0.0)
    box.SetThetaRange(0.0, 1.0)
    box.SetPhiRange(0.0, 360.0)
    box.SetEkinRange(0.6, 0.6)
    generator.AddGenerator(box)
    run.SetGenerator(generator)

    # Geometry
    cave = ROOT.R3BCave("Cave")
    cave.SetGeometryFileName("r3b_cave_vacuum.geo")
    run.AddModule(cave)

    doubleplane = 30
    neuland_position = ROOT.TGeoTranslation(0.0, 0.0, 15 * 100 + doubleplane * 10.0 / 2.0)
    neuland = ROOT.R3BNeuland(doubleplane, neuland_position)
    run.AddModule(neuland)

    # Prepare to run
    run.Init()
    ROOT.TVirtualMC.GetMC().SetRandom(ROOT.TRandom3(1337))
    ROOT.TVirtualMC.GetMC().SetMaxNStep(100000)

    # Runtime Database
    rtdb = run.GetRuntimeDb()
    parout = ROOT.FairParRootFileIo(True)
    parout.open(parfile)
    rtdb.setOutput(parout)
    rtdb.saveOutput()

    run.Run(40)

In [ ]:
simulation()

In [4]:
def digitization():
    inpfile = "output/traj.simu.root"
    parfile = "output/traj.para.root"
    outfile = "output/traj.digi.root"

    if not os.path.isfile(inpfile):
        print(f"Input {inpfile} does not exist")
        return

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("WARNING")

    run = ROOT.FairRunAna()
    run.SetSource(ROOT.FairFileSource(inpfile))
    run.SetSink(ROOT.FairRootFileSink(outfile))

    # Connect Runtime Database
    rtdb = run.GetRuntimeDb()
    pario = ROOT.FairParRootFileIo(False)
    pario.open(parfile)
    rtdb.setFirstInput(pario)
    rtdb.setOutput(pario)
    rtdb.saveOutput()

    # Digitize data to hit level and create respective histograms
    run.AddTask(ROOT.R3BNeulandDigitizer())

    # Build clusters and create respective histograms
    run.AddTask(ROOT.R3BNeulandClusterFinder())

    # Find the actual primary interaction points and their clusters
    run.AddTask(ROOT.R3BNeulandPrimaryInteractionFinder())
    run.AddTask(ROOT.R3BNeulandPrimaryClusterFinder())

    # Create spectra
    run.AddTask(ROOT.R3BNeulandMCMon())
    run.AddTask(ROOT.R3BNeulandHitMon())
    run.AddTask(ROOT.R3BNeulandClusterMon("NeulandPrimaryClusters", "NeulandPrimaryClusterMon"))
    run.AddTask(ROOT.R3BNeulandClusterMon())

    run.Init()
    run.Run(0, 0)

In [5]:
# Note: Will kill the worker if simulation ran
# digitization()

In [18]:
# Note: Will fail if simulation *didn't* run
f = ROOT.TFile.Open("output/traj.simu.root")
t = f.evt
t.AddFriend("evt", "output/traj.digi.root")

zoffset = 1500

allparticles = []
allpoints = []
allhits = []
for e in t:
    data = []
    for i, track in enumerate(e.GeoTracks):
        # Cut tracks that come much later, i.e. neutron decays
        if track.GetPoint(0)[3] > 1e-5:
            continue
        
        datum = {}
        particle = track.GetParticle()

        lv = ROOT.TLorentzVector()
        particle.Momentum(lv)
        ekin = (lv.E() - particle.GetMass()) * 1000

        if ekin < 0.1:
            continue
        
        # print(i, particle.GetPdgCode(), ekin, track.GetNpoints())
        datum["pdg"] = particle.GetPdgCode()
        datum["ekin"] = ekin

        tr = []
        for n in range(track.GetNpoints()):
            # Check bounds
            if (
                abs(track.GetPoint(n)[2] - (zoffset + 150)) > 300 / 2 * 1.1
                or abs(track.GetPoint(n)[1]) > 250 / 2 * 1.#1
                or abs(track.GetPoint(n)[0]) > 250 / 2 * 1.#1
            ):
                continue
            tr.append((track.GetPoint(n)[0] / 10, track.GetPoint(n)[1] / 10, (track.GetPoint(n)[2] - zoffset) / 10,))
        datum["track"] = tr
        data.append(datum)
    allparticles.append(data)

    points = []
    for p in e.NeulandPoints:
        pos = p.GetPosition()
        points.append((pos.X() / 10, pos.Y() / 10, (pos.Z() - zoffset) / 10, p.GetLightYield()))
    allpoints.append(points)

    hits = []
    for h in e.NeulandHits:
        pos = h.GetPosition()
        hits.append((pos.X() / 10, pos.Y() / 10, (pos.Z() - zoffset) / 10, h.GetE()))
    allhits.append(hits)

In [19]:
import palettable

colors = palettable.colorbrewer.qualitative.Set1_9.colors

particles = {
    11: {"name": "$e^-$", "color": colors[3], "line": "dashdotted"},
    -11: {"name": "$e^+$", "color": colors[3], "line": "dashdotted"},
    -12: {"name": "$\\bar{\\nu}_e$", "color": colors[4], "line": "loosely dashdotted"},
    12: {"name": "$\\nu_e$", "color": colors[4], "line": "loosely dashdotted"},
    13: {"name": "$\\mu^-$", "color": colors[7], "line": "loosely dashdotted"},
    -13: {"name": "$\\mu^+$", "color": colors[7], "line": "loosely dashdotted"},
    -14: {"name": "$\\bar{\\nu}_\\mu$", "color": colors[4], "line": "loosely dashdotted"},
    14: {"name": "$\\nu_\\mu$", "color": colors[4], "line": "loosely dashdotted"},
    22: {"name": "$\\gamma$", "color": colors[2], "line": "dotted"},
    111: {"name": "$\\pi^0$", "color": colors[5], "line": "loosely dashdotted"},
    211: {"name": "$\\pi^+$", "color": colors[5], "line": "loosely dashdotted"},
    -211: {"name": "$\\pi^-$", "color": colors[5], "line": "loosely dashdotted"},
    2112: {"name": "Neutron", "color": colors[0], "line": "densely dotted"},
    2212: {"name": "Proton", "color": colors[1], "line": "solid"},
    1000010020: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000010030: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000020030: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000020040: {"name": "$\\alpha$", "color": colors[6], "line": "solid"},
    1000020060: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000030060: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000030070: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000040070: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000040090: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000040100: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000050100: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000050110: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000060100: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000060110: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000060119: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000060120: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000060129: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000060130: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000070120: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000090180: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000110230: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000120250: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000120260: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000130260: {"name": "XXX", "color": colors[8], "line": "solid"},
    1000130270: {"name": "XXX", "color": colors[8], "line": "solid"},
}

In [20]:
import math
import matplotlib

cmap = matplotlib.cm.get_cmap("viridis_r")

for en in range(40):
    for mode in [""]:  # "l", "n"
        out = """\\documentclass{scrartcl}
        \\usepackage{xcolor, tikz}
        \\usepackage{pgfplots}
        \\pgfplotsset{compat=newest}
        \\pagestyle{empty}
        """

        cl = set([abs(datum["pdg"]) for datum in allparticles[en]])
        for pdg in cl:
            out += "\\definecolor{pdg%s}{RGB}{%d,%d,%d}\n" % (str(abs(pdg)), *particles[pdg]["color"])

        out += "\\begin{document}\n"
        out += "\\begin{tikzpicture}[scale=0.4] %\columnwidth/252.0pt]\n"

        for i in range(6):
            out += "\\draw[step=0.5,very thin,gray] (%f,-12.499) grid (%f,3.25);\n" % (i - 0.001, i + 0.5)  # 5.25
        for i in range(6, 30):
            out += "\\draw[step=0.5,very thin,gray] (%f,-12.499) grid (%f,12.499);\n" % (i - 0.001, i + 0.5)
        out += "\\draw[very thin,gray] (0,-12.5) -- (30,-12.5) -- (30,12.5) -- (6,12.5);\n"

        maxe = 50  # max([hit[3] for hit in allhits[en]])
        for hit in allhits[en]:
            c = cmap(hit[3] / maxe)
            out += "\\definecolor{tempcolor}{rgb}{%f,%f,%f}" % (c[0], c[1], c[2])
            out += "\\draw[fill=tempcolor,fill opacity=1] (%f,%f) rectangle (%f,%f);\n" % (
                hit[2] - 0.25,
                hit[1] - 0.25,
                hit[2] + 0.25,
                hit[1] + 0.25,
            )

        for point in allpoints[en]:
            if mode == "l":
                out += "\\draw (%f,%f) circle (%f);\n" % (point[2], point[1], 0.1 * math.log(point[3]))
            if mode == "n":
                out += "\\draw (%f,%f) circle (%f);\n" % (point[2], point[1], 100 * point[3])

        for datum in allparticles[en]:
            out += "\\draw[color=%s, line width=3pt, %s] " % (
                "pdg%s" % str(abs(datum["pdg"])),
                particles[datum["pdg"]]["line"],
            )
            out += " -- ".join([f"({t[2]}, {t[1]})" for t in datum["track"]])
            out += ";\n"

        legend = set([datum["pdg"] for datum in allparticles[en]])
        i = 1
        hasIon = False
        for pdg in legend:
            if particles[pdg]["name"] != "XXX":
                out += "\\draw[color=%s, very thick, %s] (0.25,%d) -- (1.25,%d) node [right,black] {%s};\n" % (
                    "pdg%s" % str(abs(pdg)),
                    particles[pdg]["line"],
                    14 - 1.1 * i,
                    14 - 1.1 * i,
                    particles[pdg]["name"],
                )
                i += 1
            else:
                if not hasIon:
                    out += "\\draw[color=%s, very thick, %s] (0.25,%d) -- (1.25,%d) node [right,black] {%s};\n" % (
                        "pdg%s" % str(abs(pdg)),
                        particles[pdg]["line"],
                        14 - 1.1 * i,
                        14 - 1.1 * i,
                        "Ions",
                    )
                    i += 1
                    hasIon = True

        out += """
        \\begin{axis}[%
            at={(0.25cm,4.75cm)},
            hide axis,
            scale only axis,
            height=0pt,
            width=0pt,
            colormap={reverse viridis}{
                indices of colormap={
                \pgfplotscolormaplastindexof{viridis},...,0 of viridis}
            },
            colorbar horizontal,
            point meta min=0,
            point meta max=50,
            colorbar style={
                width=5cm,
                xtick={50, 25, 0},
            }]
        \\end{axis}
        """

        out += """
        \\end{tikzpicture}
        \\end{document}
        """

        with open(f"results/tracks/tracks{en:02d}{mode}.tex", "w") as f:
            f.write(out)

        import subprocess

        subprocess.call(["lualatex", f"tracks{en:02d}{mode}.tex"], cwd="results/tracks", stdout=subprocess.DEVNULL)
        subprocess.call(
            ["pdfcrop", f"tracks{en:02d}{mode}.pdf", f"tracks{en:02d}{mode}.pdf"],
            cwd="results/tracks",
            stdout=subprocess.DEVNULL,
        )
        os.remove(f"results/tracks/tracks{en:02d}{mode}.aux")
        os.remove(f"results/tracks/tracks{en:02d}{mode}.log")

In [ ]:
# Note: Will fail if simulation *didn't* run
file = ROOT.TFile.Open("output/traj.simu.root")
tree = file.evt

nevent = -1
targetevent = 1
data = []
for event in tree:
    nevent += 1
    if nevent != targetevent:
        continue
  
    for i, track in enumerate(event.GeoTracks):
        particle = track.GetParticle()
        lv = ROOT.TLorentzVector()
        particle.Momentum(lv)
        ekin = (lv.E() - particle.GetMass()) * 1000

        data.append((i, particle.GetFirstMother(), particle.GetPdgCode(), track.GetName(), round(ekin), track.GetPoint(0)[3]))
        
data = pd.DataFrame(data)
data.columns = ["ID", "Mother-ID", "PDG", "Name", "Ekin", "Starttime"]
display(data)